In [1]:
from pytorch_lightning.callbacks import ModelCheckpoint#, DeviceStatsMonitor
from pytorch_lightning import Trainer
#from pytorch_lightning.profiler import SimpleProfiler
from pytorch_lightning.loggers import TensorBoardLogger
from dl_toolbox.lightning_modules import *
from dl_toolbox.lightning_datamodules import *
from pathlib import Path, PurePath

/d/pfournie/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trainer = Trainer(
    max_steps=100000,
    gpus=1,
    multiple_trainloader_mode='min_size',
    limit_train_batches=1.,
    limit_val_batches=1.,
    logger=TensorBoardLogger(
        #save_dir='/data/outputs',
        save_dir='/work/OT/ai4usr/fournip/outputs',
        name='digiv2paris',
        version='ce_d4color3',
        sub_dir='0'
    ),
    #profiler=SimpleProfiler(),
    callbacks=[
        ModelCheckpoint(),
        #DeviceStatsMonitor(),
    ],
    num_sanity_val_steps=0,
    check_val_every_n_epoch=1,
    benchmark=True,
    enable_progress_bar=True
)

/d/pfournie/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..


In [3]:
datamodule = Splitfile(
    epoch_len=1000,
    batch_size=16,
    workers=4,
    splitfile_path=Path.home() / f'ai4geo/splits/digitanieV2/paris.csv',
    test_folds=(9,),
    train_folds=tuple(range(9)),
    data_path=Path('/work/OT/ai4geo/DATA/DATASETS/DIGITANIE'),
    crop_size=256,
    img_aug='d4_color-3',
    unsup_img_aug=None,
    labels='6',
    unsup_train_folds=None
)

Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_0.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_1.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_3.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_4.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_5.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154_6.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Paris/Paris_EPSG2154

In [4]:
module = CE(
    ignore_index=-1,
    #no_pred_zero=False,
    #mixup=0.4,
    network='SmpUnet',
    encoder='efficientnet-b1',
    pretrained=False,
    weights=[],
    in_channels=3,
    out_channels=6,
    initial_lr=0.001,
    final_lr=0.0005,
    plot_calib=True,
    class_names=datamodule.class_names
    #alphas=(0., 1.),
    #ramp=(0, 40000),
    #pseudo_threshold=0.9,
    #consist_aug='color-3',
    #emas=(0.9, 0.999)
)

In [5]:
trainer.fit(
    model=module,
    datamodule=datamodule,
    ckpt_path=None
)

/work/OT/ai4usr/fournip/vre_venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /work/OT/ai4usr/fournip/outputs/digiv2paris/ce_d4color3/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | network | SmpUnet          | 8.8 M 
1 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.031    Total estimated model params size (MB)


Epoch 0:  94%|█████████▍| 62/66 [00:34<00:02,  1.81it/s, loss=1.71, v_num=lor3]
Validation: 0it [00:00, ?it/s]
Epoch 0:  97%|█████████▋| 64/66 [00:37<00:01,  1.71it/s, loss=1.71, v_num=lor3]

/work/OT/ai4usr/fournip/vre_venv/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(



Epoch 1:  23%|██▎       | 15/66 [00:08<00:27,  1.83it/s, loss=1.69, v_num=lor3]

/work/OT/ai4usr/fournip/vre_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
